In [300]:
from surprise import KNNWithMeans, SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import Ridge

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split as train_test_split_s
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error

### Гибридная рекомендательная система
Я решил попробовать сделать рекомендательную систему каскадного типа с примесью блендинга.
Первой моделью я выбираю 20 наиболее подходящих рекомендаций, второй моделью их ранжирую другим способом, и в итоге смешиваю результат, отдавая большее предпочтение второй модели.

данные для обучения первой модели:

In [301]:
    movies = pd.read_csv('D:/GitRepo/Нетология/Рекомендательные системы/netology-recsys-master/netology-recsys-master/lecture-1/movies.csv')
    ratings = pd.read_csv('D:/GitRepo/Нетология/Рекомендательные системы/netology-recsys-master/netology-recsys-master/lecture-1/ratings.csv')
    
    movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
    movies_with_ratings.dropna(inplace=True)
    
    print('shape', movies_with_ratings.shape)
    
    dataset = pd.DataFrame({
        'uid': movies_with_ratings.userId,
        'iid': movies_with_ratings.movieId,
        'rating': movies_with_ratings.rating
    })
    
    print('users:', len(dataset.uid.unique()))
    print('items:',len(dataset.iid.unique()))
    
    
    r_min = ratings.rating.min()
    r_max = ratings.rating.max()
    
    reader = Reader(rating_scale=(r_min, r_max))
    data = Dataset.load_from_df(dataset, reader)
    
    trainset, testset = train_test_split(data, test_size=.15)

shape (100836, 6)
users: 610
items: 9724


Первая модель - SVD

In [310]:
algo = SVD(n_factors=20, n_epochs=30)
algo.fit(trainset)

In [311]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8695


0.8695282854721408

Вторая модель - линейное предсказание оценки, используя разные фичи фильмов и пользователей

In [314]:
def change_string(s):
    return ' '.join(str(s).replace(' ', '').replace('-', '').split('|'))

Загружаем и преобразовываем данные для обучения второй модели

In [315]:
    movies = pd.read_csv('D:/GitRepo/Нетология/Рекомендательные системы/netology-recsys-master/netology-recsys-master/lecture-1/movies.csv')
    print('---10---')
    ratings = pd.read_csv('D:/GitRepo/Нетология/Рекомендательные системы/netology-recsys-master/netology-recsys-master/lecture-1/ratings.csv')
    print('---9---')
    tags = pd.read_csv('D:/GitRepo/Нетология/Рекомендательные системы/netology-recsys-master/netology-recsys-master/lecture-1/tags.csv')
    print('---8---')
    # Users
    df_1 = ratings[['userId', 'movieId', 'rating']]
    df_1_1 = pd.DataFrame(df_1.groupby('userId').rating.mean()).rename(columns={'rating': 'user_mean_rating'})
    df_1_1 = pd.merge(df_1_1, pd.DataFrame(df_1.groupby('userId').rating.median()).rename(columns={'rating': 'user_median_rating'}),
             left_index=True,
             right_index=True)
    df_1_1 = pd.merge(df_1_1, pd.DataFrame(df_1.groupby('userId').rating.var()).rename(columns={'rating': 'user_variance_rating'}),
             left_index=True,
             right_index=True)
    df_1_1 = pd.merge(df_1_1, pd.DataFrame(df_1.groupby('userId').rating.max()).rename(columns={'rating': 'user_max_rating'}),
             left_index=True,
             right_index=True)
    df_1_1 = pd.merge(df_1_1, pd.DataFrame(df_1.groupby('userId').rating.min()).rename(columns={'rating': 'user_min_rating'}),
             left_index=True,
             right_index=True)
    print('---7---')
    
    # Movies
    df_1_2 = pd.DataFrame(df_1.groupby('movieId').rating.mean()).rename(columns={'rating': 'movie_mean_rating'})
    df_1_2 = pd.merge(df_1_2, pd.DataFrame(df_1.groupby('movieId').rating.median()).rename(columns={'rating': 'movie_median_rating'}),
             left_index=True,
             right_index=True)
    df_1_2 = pd.merge(df_1_2, pd.DataFrame(df_1.groupby('movieId').rating.var()).rename(columns={'rating': 'movie_variance_rating'}),
             left_index=True,
             right_index=True).fillna(0)
    df_1_2 = pd.merge(df_1_2, pd.DataFrame(df_1.groupby('movieId').rating.max()).rename(columns={'rating': 'movie_max_rating'}),
             left_index=True,
             right_index=True)
    df_1_2 = pd.merge(df_1_2, pd.DataFrame(df_1.groupby('movieId').rating.min()).rename(columns={'rating': 'movie_min_rating'}),
             left_index=True,
             right_index=True)
    print('---6---')
    
    # GENRES
    df_2 = movies[['movieId', 'genres']]
    movie_genres = [change_string(g) for g in df_2.genres.values]
    count_vect = CountVectorizer()
    buffer = count_vect.fit_transform(movie_genres)
    tfidf_transformer  = TfidfTransformer()
    buffer = tfidf_transformer.fit_transform(buffer)
    df_2 = pd.DataFrame(buffer.toarray(), columns=count_vect.get_feature_names())
    df_2 = pd.merge(movies[['movieId']], df_2, how='left', left_index=True, right_index=True)
    df_2.index = df_2.movieId
    df_2.drop(columns=['movieId'], inplace=True)
    df_2.fillna(0.0, inplace=True)
    print('---5---')
    
    # TAGS
    movie_gtags = [change_string(g) for g in tags.tag.values]
    count_vect = CountVectorizer()
    buffer = count_vect.fit_transform(movie_gtags)
    buffer1 = pd.DataFrame(buffer.toarray(), columns=count_vect.get_feature_names())
    common_tags = buffer1.sum().sort_values(ascending=False)[buffer1.sum().sort_values(ascending=False) > 5].index
    tfidf_transformer  = TfidfTransformer()
    buffer = tfidf_transformer.fit_transform(buffer)
    df_3 = pd.DataFrame(buffer.toarray(), columns=count_vect.get_feature_names())
    df_3 = df_3[common_tags]
    df_3 = pd.merge(movies[['movieId']], df_3, how='left', left_index=True, right_index=True)
    df_3.index = df_3.movieId
    df_3.drop(columns=['movieId'], inplace=True)
    df_3.fillna(0.0, inplace=True)
    print('---4---')
    
    # final dataset     
    df = ratings[['userId', 'movieId', 'rating']]
    df = pd.merge(df, df_1_1, how='left', left_on='userId', right_index=True)
    df = pd.merge(df, df_1_2, how='left', left_on='movieId', right_index=True)
    df = pd.merge(df, df_2, how='left', left_on='movieId', right_index=True)
    df = pd.merge(df, df_3, how='left', left_on='movieId', right_index=True)
    print('---3---')
    
    scaler = StandardScaler()
    X = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    print('---2---')
    
    X_train, X_test, y_train, y_test = train_test_split_s(X.drop(columns=['rating']), 
                                                        X['rating'], 
                                                        test_size=0.1)
    print('---1---')
    print('---0---')

---10---
---9---
---8---
---7---
---6---
---5---
---4---
---3---
---2---
---1---
---0---


обучаем модель

In [316]:
params = {'alpha':list(np.arange(0.1, 100.0, 0.1)),
          'max_iter':[500, 1000, 2000, 5000, 10000],
          'tol':[1e-3, 1e-2, 1e-4, 1e-5],
          'solver':['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']          
          }
linear_model = RandomizedSearchCV(Ridge(), params, cv=10, random_state=13, n_iter=10)
linear_model = linear_model.fit(X_train,y_train).best_estimator_
linear_model

Ridge(alpha=18.1, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, random_state=None, solver='lsqr', tol=0.001)

In [318]:
print('RMSE:', mean_squared_error(y_test, linear_model.predict(X_test), squared=False))

RMSE: 0.7752806599372357


Предсказываем 5 рекомендованных фильмов для пользователя:

In [366]:
def recomendation_for_user(user_id):
    user_movies = df[df.userId == user_id].movieId.unique()
    m = df[~df.movieId.isin(user_movies)][movie_features+['movieId']].drop_duplicates()
    m['userId'] = user_id
    m['rating'] = 0

    m = pd.merge(m, df[df.userId==user_id][user_features+['userId']].drop_duplicates(), how='left', left_on='userId', right_on='userId')
    m = m[X.columns]
    to_model = pd.DataFrame(scaler.transform(m), columns=m.columns)

    res_1 = pd.concat([to_model.drop(columns=['rating']), pd.Series(linear_model.predict(to_model.drop(columns=['rating'])), name='rating')], axis=1)

    res_1 = pd.DataFrame(scaler.inverse_transform(res_1[X.columns]), columns=X.columns).sort_values('rating', ascending=False).head(20)[['movieId','rating']].reset_index(drop=True)

    rats = []
    for mov in res_1.movieId:
        rats.append({'movieId': algo.predict(uid=user_id, iid=mov).iid,
                    'rating': algo.predict(uid=user_id, iid=mov).est})
    res_2 = pd.merge(res_1, pd.DataFrame(rats), how='left', left_on='movieId', right_on='movieId')

    res_2['rating_z'] = (res_2['rating_x']/5 + res_2['rating_y']/3)
    recommendation = res_2.sort_values('rating_z', ascending=False).head(5).movieId
#     print(recommendation)
    for n,i in enumerate(recommendation):
        print(n+1,':',movies[movies.movieId==i].title.iloc[0])
    

Проверяем для трех разных пользователей:

In [367]:
recomendation_for_user(2.0)

1 : Jonah Who Will Be 25 in the Year 2000 (Jonas qui aura 25 ans en l'an 2000) (1976)
2 : Babes in Toyland (1934)
3 : Unfaithfully Yours (1948)
4 : Calcium Kid, The (2004)
5 : Denise Calls Up (1995)


In [368]:
recomendation_for_user(50.0)

1 : Jonah Who Will Be 25 in the Year 2000 (Jonas qui aura 25 ans en l'an 2000) (1976)
2 : Unfaithfully Yours (1948)
3 : Siam Sunset (1999)
4 : Calcium Kid, The (2004)
5 : Satin Rouge (2002)


In [369]:
recomendation_for_user(550.0)

1 : Jonah Who Will Be 25 in the Year 2000 (Jonas qui aura 25 ans en l'an 2000) (1976)
2 : Go for Zucker! (Alles auf Zucker!) (2004)
3 : Sorority House Massacre II (1990)
4 : Babes in Toyland (1934)
5 : Denise Calls Up (1995)


Да, безусловно в работе этой системы есть странности (например почему-то на первом месте в рекомендации дается один фильм и подобное). Но эта модель и не претендует на что-то суперкачественное :) 
Но в качестве модельной гибридной системы, думаю, подойдет :)